In [3]:
class RefPath:
    INITIAL = "#/info/x-clusters/"
    WORKER_NODES = "worker-nodes"
    PODS = "pods"
    CONTAINERS = "containers"
    INFO = "info"
    X_CLUSTERS = "x-clusters"
    X_LOCATION = "x-location"
    REF = "$ref"
    PATHS = "paths"
    
    def __init__(self, *arg):
        if len(arg) == 1:            
            paths = arg[0].split("#/info/x-clusters/")
            parts = paths[1].split("/")        
            self.cluster = parts[0]
            self.worker_node = parts[2]
            self.pod_name = parts[4]
            self.container_name = parts[6]
        else:
            self.cluster = arg[0]
            self.worker_node = arg[1]
            self.pod_name = arg[2]
            self.container_name = arg[3]    
        

    @property
    def full_path(self):
        return f"{RefPath.INITIAL}{self.cluster}/{RefPath.WORKER_NODES}/{self.worker_node}/{RefPath.PODS}/{self.pod_name}/{RefPath.CONTAINERS}/{self.container_name}"


class Method:
    def __init__(self, path_name, method_name, ref_path, load, schema_name, full_method):
        self.path_name = path_name
        self.method_name = method_name
        self.ref_path = ref_path      
        self.load = load
        self.schema_name = schema_name        
        self.full_method = full_method
        self.contribution = 0

    def __str__(self):
        return self.path_name + ": " + self.method_name + " (" + self.ref_path.full_path + ") " + str(self.load) + " <-> " + self.schema_name
    
class Component:
    def __init__(self, name, load, full_component, ref_path=None):
        self.name = name
        self.load = load
        self.full_component = full_component
        self.ref_path = ref_path
        
    def __str__(self):
        return self.name + " ---- " + str(self.load) + " (" + self.ref_path + ")"
    
import operator
class Group:
    def __init__(self, parent_load, sum_of_loads, components):        
        self.parent_load = parent_load        
        self.sum_of_loads = sum_of_loads
        self.components = components
        
    def get_contributed_components(self):
        components = []
        for component in self.components:
            component.contribution = (component.load / self.sum_of_loads) * self.parent_load
            components.append(component)        
        return sorted(components, key=operator.attrgetter("contribution"))


In [4]:
class Cluster(Component):
    def __init__(self, pod_name, load, full_component, ref_path):        
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_cluster = True
        
    @property
    def worker_nodes(self):
        worker_nodes = []
        for wn in self.full_component["worker-nodes"].values():
            ref_path = self.ref_path + "/worker-nodes/" + wn["name"]
            worker_nodes.append(WorkerNode(wn["name"], wn["metrics"]["load"], wn, ref_path))
        return worker_nodes
        
class WorkerNode(Component):
    def __init__(self, pod_name, load, full_component, ref_path):
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_worker_node = True
        
    @property
    def pods(self):
        pods = []
        for pod in self.full_component["pods"].values():
            ref_path = self.ref_path + "/pods/" + pod["name"]
            pods.append(Pod(pod["name"], pod["metrics"]["load"], pod, ref_path))        
        return pods
        
class Pod(Component):
    def __init__(self, pod_name, load, full_component, ref_path):
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_pod = True
        self.contribution = 0
        
    @property
    def containers(self):
        containers = []
        for container_name, container in self.full_component["containers"].items():
            ref_path = self.ref_path + "/containers/" + container_name
            containers.append(Container(container_name, container["metrics"]["load"], container, ref_path))        
        return containers
        
class Container(Component):
    def __init__(self, pod_name, load, full_component, ref_path):
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_container = True


class PodGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_pod_group = True            
        
    def __str__(self):
        return "PodGroup sum_of_pod_loads: " + str(sum_of_pod_loads)
    
class ContainerGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_container_group = True            
        
    def __str__(self):
        return "ContainerGroup sum_of_pod_loads: " + str(sum_of_pod_loads)
    
class MethodGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_method_group = True            
        
    def __str__(self):
        return "ContainerGroup sum_of_pod_loads: " + str(sum_of_pod_loads)
        

In [5]:
def _gen_dict_extract(key, var):
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):                
                for result in _gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, (list)):
                for d in v:
                    for result in _gen_dict_extract(key, d):
                        yield result                        


def _get_schema_only(references):
    saved_schema = 'default'
    for reference in references:
        schema = reference.split("#/components/schemas/")        
        try:
            saved_schema = schema[1]
        except IndexError:
            pass
    
    return saved_schema


def _join_components(threshold, components):    
    joined_components = []
    remaining_components = []
    collective_contribution = 0
    for i in range(len(components)):
        component = components[i]
        collective_contribution += component.contribution
        
        if collective_contribution < threshold:
            joined_components.append(component)
        else:
            remaining_components.append(component)
        
    return joined_components, remaining_components

In [30]:
# Monitor Scaling start from here
import json

with open("01data.json") as datafile:
    all_data = json.load(datafile)
    

with open("01config.json") as config_file:
    config_template = json.load(config_file)

In [31]:
import copy

POD_LEVEL = "pod"
CL_LEVEL = "cluster"
WN_LEVEL = "worker-node"

copied_template = copy.deepcopy(config_template)

config_paths = config_template["paths"]
methods = []
for path_name, path in config_paths.items():
    for method_name, method in path.items():
        ref_path = RefPath(method["x-location"]["$ref"])
        
        all_references = list(set(_gen_dict_extract('$ref', method)))
        schema_name = _get_schema_only(all_references)

        methods.append(Method(path_name, method_name, ref_path, method["x-metrics"]["load"], schema_name, method))

method_schemas = {}
for method in methods:    
    method_schemas.setdefault(method.schema_name, {"pods": set(), "worker-nodes": set()})    
    method_schemas[method.schema_name]["pods"].add(method.ref_path.pod_name)
    method_schemas[method.schema_name]["worker-nodes"].add(method.ref_path.worker_node)
    

# it will remove the schema of stateless methods
if 'default' in method_schemas.keys():
    del method_schemas['default']

final_schema_levels = dict()
for schema_name, values in method_schemas.items():    
    if len(values["worker-nodes"]) > 1:
        final_schema_levels[schema_name] = CL_LEVEL
    elif len(values["pods"]) > 1:
        final_schema_levels[schema_name] = WN_LEVEL
    else:
        final_schema_levels[schema_name] = POD_LEVEL
        
config_schemas = copied_template['components']['schemas']
for schema_name, level in final_schema_levels.items():
    if schema_name in config_schemas.keys():
        config_schemas[schema_name]['x-storage-level'] = level

config_schemas

{'Order': {'x-storage-level': 'pod',
  'type': 'object',
  'properties': {'id': {'type': 'integer', 'format': 'int64'},
   'petId': {'type': 'integer', 'format': 'int64'},
   'quantity': {'type': 'integer', 'format': 'int32'},
   'shipDate': {'type': 'string', 'format': 'date-time'},
   'status': {'type': 'string',
    'description': 'Order Status',
    'enum': ['placed', 'approved', 'delivered']},
   'complete': {'type': 'boolean', 'default': False}},
  'xml': {'name': 'Order'}},
 'Category': {'x-storage-level': 'pod',
  'type': 'object',
  'properties': {'id': {'type': 'integer', 'format': 'int64'},
   'name': {'type': 'string'}},
  'xml': {'name': 'Category'}},
 'User': {'x-storage-level': 'worker-node',
  'type': 'object',
  'properties': {'id': {'type': 'integer', 'format': 'int64'},
   'username': {'type': 'string'},
   'firstName': {'type': 'string'},
   'lastName': {'type': 'string'},
   'email': {'type': 'string'},
   'password': {'type': 'string'},
   'phone': {'type': 'strin